In [ ]:
import csv
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt 
from sklearn.neighbors import KDTree
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import ipywidgets as widgets # for interactive sliders
import pickle


%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import *

In [ ]:
DATASET_DIR = "/home/ori/git/realtime-trees/single_trees/clustering_2/"
SLICE_HEIGHT = 0.5
TREE_ID = 63

In [ ]:
file_name = DATASET_DIR + "tree" + str(TREE_ID).zfill(3) + ".pkl"
with open(file_name, 'rb') as file:
    cluster = pickle.load(file)
cluster

In [ ]:
file_name = DATASET_DIR + "tree" + str(TREE_ID).zfill(3) + ".pkl"
with open(file_name, 'rb') as file:
    cluster = pickle.load(file)

tree = Tree.reconstruct(
    cluster,
    SLICE_HEIGHT,
    slice_thickness=0.3,
    
    save_points=True,
    save_debug_results=True,
    save_votes=True,
    max_consecutive_fails=5,
    filter_min_points=10,
    min_hough_vote=0.1,

)
fig = go.Figure()
points = cluster["cloud"].point.positions.numpy()
if len(tree.circles) != 0:
    all_slice_points = np.concatenate(tree.points)
    # points = all_slice_points

    plt.figure()
    @widgets.interact(i=widgets.IntSlider(min=0, max=len(tree.points)-1, step=1, value=0))
    def plot_ith_slice(i):
        # show two plots side by side, one for points, one for the mask
        fig, ax = plt.subplots(1,2, figsize=(15,6))
        
        # scatter plot and circles
        ax[0].set_title('Points')
        ax[0].set_aspect('equal')
        ax[0].set_xlim(np.min(all_slice_points[:,0]), np.max(all_slice_points[:,0]))
        ax[0].set_ylim(np.min(all_slice_points[:,1]), np.max(all_slice_points[:,1]))
        # scatter all points
        ax[0].scatter(tree.points[i][:,0], tree.points[i][:,1], s=1, color="r", label='All Points')
        ax[0].scatter(tree.hough_points[i][:,0], tree.hough_points[i][:,1], s=5, color="g", label='Filtered Points')
        # plot circles
        ax[0].add_artist(plt.Circle((tree.circles[i].x, tree.circles[i].y), tree.circles[i].radius, color='g', fill=False, label=f'Circle'))
        ax[0].add_artist(plt.Circle((tree.hough_circles[i].x, tree.hough_circles[i].y), tree.hough_circles[i].radius, color='r', fill=False, label=f'Hough Circle'))
        print((tree.hough_circles[i].x, tree.hough_circles[i].y), tree.hough_circles[i].radius)
        ax[0].legend(loc='upper right')
        
        #  mask
        ax[1].set_title('Pixels for Hough Transformation')
        print(tree.hough_votes[i].min(), tree.hough_votes[i].max())
        ax[1].imshow(np.rot90(tree.hough_votes[i], k = -1), vmin=0, vmax=5)
        # ax[1].invert_yaxis()

    for i in range(len(tree.circles) - 1):
        tree.circles[i].plot(fig, color='red')
        tree.circles[i].plot_cone_frustum(tree.circles[i + 1], fig)
        
fig.add_trace(go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=2, color='black')))
fig.update_scenes(aspectmode='data')
fig.update_layout( autosize=False, width=600, height=800, margin=dict(l=1,r=1,b=1,t=1,pad=4))
fig.show()

In [ ]:
import trimesh
import os
dir = 'single_trees/clustering/'
for file in os.listdir(dir):
    if "pcd" not in file:
        continue
    points = np.asarray(o3d.io.read_point_cloud(os.path.join(dir, file)).points)
    tree = Tree.reconstruct(
        points,
        SLICE_HEIGHT,
        slice_thickness=0.2,
        save_points=True,
        save_debug_results=True,
        save_votes=True
    )
    verts, tris = tree.generate_mesh()
    if tris.shape[0] == 0:
        continue
    mesh = trimesh.Trimesh(vertices=verts, faces=tris)
    mesh.export(os.path.join('single_trees/clustering_map/meshes', file.replace('.pcd', '.obj')))